In [ ]:
import shutil
shutil.rmtree("/content/sample_data")

In [ ]:
! pip install mediapipe
! pip install sk-video
import numpy as np 
import pandas as pd 
import os
import skvideo.io
import cv2 
import mediapipe as mp
import time
import matplotlib.pyplot as plt

In [ ]:
! wget https://zenodo.org/record/4010759/files/Greetings_2of2.zip

In [ ]:
! unzip Greetings_2of2.zip

In [ ]:
os.makedirs("/content/Input")

In [ ]:
wordtype=["Greetings"]

for i in wordtype:
   for dirpath,dirfile,filename in os.walk(os.path.join("/content",i)):
     print(dirfile)

In [ ]:
d=['/content/Greetings']
target_dir = '/content/Input'

for i in d:
    file_names = os.listdir(i)
    for file_name in file_names:
        shutil.move(os.path.join(i, file_name), target_dir)

In [ ]:
os.makedirs("/content/Output")
#no need face pipe

In [ ]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic
##Holistic
def holistic_estimation(image,results):
        
        # 1. Draw face landmarks
  #      mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
  #                               mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=2),
  #                               mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=2)
  #                               )rgb(204, 0, 0)
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(102,255,51),thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)
                               )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec( color=(102,255,51),thickness=3, circle_radius=4),
                                 mp_drawing.DrawingSpec( color=(255,255,255),thickness=2, circle_radius=2) 
                                )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec( color=(255, 255, 0),thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec( color=(255,255,255),thickness=2, circle_radius=2)
                                 )
                        
        return image

In [ ]:
def video_array_maker(pather,height=224,width=224):
  videodata = skvideo.io.vread(pather)  
 # print(videodata[0].shape)
  outpath=os.path.join("./Output",pather.split("/")[3],os.path.split(pather)[1])
  out = cv2.VideoWriter(outpath,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
  start=time.time()

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#    arr=[]
    #actualframe=len(videodata)
    #frames=len(videodata)
    #print("Actual frame {}".format(actualframe))
    if actualframe >=45:
          for i in range (actualframe):
            x=round (actualframe/(45)  * i)
            if x >=actualframe:
                    break
            else:
                frame =videodata[x]
                #frame =cv2.resize(frame,(720,1080),interpolation=cv2.INTER_AREA)    # --> Use this If U have resize and send to the model  (0.2 Seconds Difference)
                results = holistic.process(frame)  

                output = holistic_estimation(frame,results)
                output =cv2.resize(output,(width,height),interpolation=cv2.INTER_AREA)
                output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
#                arr.append(output)
                out.write(output)               
    else:
          for i in range(actualframe):
              frame=videodata[i]
              frame=cv2.resize(frame,(width,height),interpolation=cv2.INTER_AREA)
              results = holistic.process(frame)  
              output = holistic_estimation(frame,results)
              output =cv2.cvtColor(output,cv2.COLOR_BGR2RGB)
              out.write(output)
          for i in range(45-actualframe):
              
              newframe=np.zeros(shape=(height,width,3))
              

              out.write(np.uint8(newframe))
    end=time.time()
    print(end-start)
    out.release()
    #print("File Created : {}".format(outpath))
    #os.remove(pather)

In [ ]:
for dirpathed,dirnamed,file in os.walk("/content/Input"):
    for class_  in dirnamed:
      os.makedirs(os.path.join("/content/Output",class_))

In [ ]:
video_array_maker("/content/Input/56. Pleased/MVI_0011.MOV")

Actual frame 53
3.8190877437591553


In [ ]:
z=0
for dirpathed,dirnamed,file in os.walk("/content/Input"):
 for class_  in dirnamed:
   for dirpath,dirname,files in os.walk(os.path.join(dirpathed,class_)):
     for i in files:
          pather=os.path.join(dirpath,i)
          z+=1
          print("..{} th Video..".format(z))
          video_array_maker(pather)
   print(".............{} over..........".format(class_))
   break